# Text

In [2]:
import pandas as pd

df = pd.read_parquet('data/ml_data.parquet')


In [3]:
df

,inputs,target
0,"[12, 6, 23, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,...",18
1,"[12, 6, 22, 19, 20, 21, 22, 23, 8, 24, 11, 11,...",12
2,"[12, 6, 21, 30, 31, 32, 33, 34, 23, 35, 8, 36,...",34
3,"[12, 6, 21, 44, 4, 45, 46, 6, 47, 8, 48, 11, 1...",20
4,"[12, 6, 20, 55, 56, 57, 11, 58, 59, 4, 60, 61,...",3
...,...,...
181111,"[12, 4, 6, 2223, 1518, 12, 2659, 11, 8, 9887, ...",1
181112,"[12, 4, 6, 55964, 1816, 11, 191, 185, 8799, 24...",39
181113,"[12, 4, 6, 48567, 11, 11, 1150, 65, 454, 607, ...",25
181114,"[12, 4, 5, 1560, 7775, 171, 10598, 3480, 171, ...",20
